# Задача 3.3

Выборка: [Wine](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html).

Предложить метод выбора наиболее важных признаков для логистической регрессии на основе изученных методов прикладной статистики. Осуществить выбор.

In [28]:
from sklearn.datasets import load_wine
import statsmodels.api as sm
import numpy as np

In [30]:
# If True, returns (data, target) instead of a Bunch object.
data = load_wine()
print(data['DESCR'])

X, y = load_wine(return_X_y=True)

.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

    :Number of Instances: 178 (50 in each of three classes)
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- Alcohol
 		- Malic acid
 		- Ash
		- Alcalinity of ash  
 		- Magnesium
		- Total phenols
 		- Flavanoids
 		- Nonflavanoid phenols
 		- Proanthocyanins
		- Color intensity
 		- Hue
 		- OD280/OD315 of diluted wines
 		- Proline

    - class:
            - class_0
            - class_1
            - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash:            1

In [29]:
import statsmodels.api as sm

logit_model = sm.Logit(endog=np.hstack(y), exog=X).fit()
logit_model.summary()

ValueError: endog must be in the unit interval.

Попробуем отобрать кандидатов на "признаки" с помощью приращения roc_auc метрики

Будем жадно отбирать до тех пор, пока добавление предыдущего не ухудшит качество

In [9]:
n = X_train.shape[0]

In [10]:
X_train.shape[1]

13

In [11]:
idxs = []
last = 0
curr_roc = 0.1
while (last < curr_roc):
    last = curr_roc
    max_idxs = []
    for feature_idx in range(X_train.shape[1]):
        if feature_idx in idxs:
            continue
        new_ids = idxs + [feature_idx]
        
        new_X = X_train[:, new_ids]
        model = LogisticRegression(multi_class='multinomial', solver='saga', max_iter=10000).fit(new_X, y_train)
        score = roc_auc_score(y_train, model.predict_proba(new_X), multi_class='ovo', average='macro')
        
        max_idxs.append([feature_idx, score])
        
    curr_roc = sorted(max_idxs, key=lambda x: x[1], reverse=True)[0][1]
    new_id = sorted(max_idxs, key=lambda x: x[1], reverse=True)[0][0]
    idxs.append(new_id)

In [12]:
idxs

[6, 0, 10, 2, 7, 3]

In [13]:
model = LogisticRegression(multi_class='multinomial', solver='saga', max_iter=10000).fit(X_train, y_train)
score = roc_auc_score(y_train, model.predict_proba(X_train), multi_class='ovo', average='macro')
score_test = roc_auc_score(y_test, model.predict_proba(X_test), multi_class='ovo', average='macro')


print("no feature selection, roc_auc_train = {:.5f}".format(score))
print("no feature selection, roc_auc_test = {:.5f}".format(score_test))

no feature selection, roc_auc_train = 0.98577
no feature selection, roc_auc_test = 0.99851


In [14]:
new_X = X_train[:, idxs]
model = LogisticRegression(multi_class='multinomial', solver='saga', max_iter=10000).fit(new_X, y_train)
score = roc_auc_score(y_train, model.predict_proba(new_X), multi_class='ovo', average='macro')
score_test = roc_auc_score(y_test, model.predict_proba(X_test[:, idxs]), multi_class='ovo', average='macro')


print("feature selection, roc_auc_train = {:.5f}".format(score))
print("feature selection, roc_auc_test = {:.5f}".format(score_test))

feature selection, roc_auc_train = 0.99194
feature selection, roc_auc_test = 0.97130


In [15]:
bootstrap_size = 100

In [18]:
rocs = []
for i in range(1, len(idxs) + 1):
    rocs_to_append = []
    new_X = X_train[:, idxs[:i]]
    
    for _ in range(bootstrap_size):
        new_idxs = np.random.choice(n, n, replace=True)
        
        X_bootstrap = new_X[new_idxs]
        new_y = y[new_idxs]
        model = LogisticRegression(multi_class='multinomial', solver='saga', max_iter=10000).fit(X_bootstrap, new_y)
        score = roc_auc_score(new_y, model.predict_proba(X_bootstrap), multi_class='ovo', average='macro')
        rocs_to_append.append(score)
    rocs.append(rocs_to_append)